In [1]:
import numpy as np
import pandas as pd

import geopandas as gpd
import shapely

In [2]:
file_path = "./data/BND_ADM_DONG_PG.shp"

In [3]:
gdf = gpd.read_file(file_path,encoding='cp949')
gdf_latlng = gdf.to_crs(epsg=4326)

In [4]:
polygon2coords = lambda poligon: poligon.exterior.coords

coords2lng = lambda coords: list(map(lambda coord: coord[0], coords))
coords2lat = lambda coords: list(map(lambda coord: coord[1], coords))

In [5]:
df = pd.DataFrame(gdf_latlng)

In [6]:
geometry_types = list(set(df['geometry'].apply(lambda x : type(x))))
geometry_types

[shapely.geometry.multipolygon.MultiPolygon, shapely.geometry.polygon.Polygon]

In [7]:
def multipolygon2polygon(poligon):
    if type(poligon) == shapely.geometry.polygon.Polygon:
        return poligon
    elif type(poligon) == shapely.geometry.multipolygon.MultiPolygon:
        return max(poligon.geoms, key=lambda p: p.area)
    else:
        return None

In [8]:
df['geometry'] = df['geometry'].apply(lambda x : multipolygon2polygon(x))

In [9]:
mean_coord_df = pd.concat(
    [
        df.drop(columns=["geometry"]),
        df["geometry"]
        .apply(
            lambda x: {
                "lat": np.mean(coords2lat(polygon2coords(x))),
                "lng": np.mean(coords2lng(polygon2coords(x))),
            }
        )
        .apply(pd.Series),
    ],
    axis=1,
)

mean_coord_df.head()

,BASE_DATE,ADM_NM,ADM_CD,lat,lng
0,20230701,사직동,11010530,37.573599,126.970017
1,20230701,삼청동,11010540,37.584574,126.981997
2,20230701,부암동,11010550,37.599156,126.962802
3,20230701,평창동,11010560,37.608381,126.965929
4,20230701,한남동,11030740,37.536131,127.004701


In [10]:
mean_coord_df.to_csv("./mean_coord_df.csv")